# Programa de la solución de las trayectorias cuánticas para el sistema de 3 niveles.
## Distribución de periodos brillantes y obscuros 
## Distrubución de tiempos de espera
## Distribución de conteo de fotones 

## Figura del proyecto


<img src="3LAkev.png" alt="a" class="bg-primary mb-1" width="400px">


In [1]:
import qutip 
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pylab import *


### La funcion basis es para el número de estados de Fock en el espacio de Hilbert. Si es una lista, entonces el objeto resultante será un producto tensorial  sobre espacios con esas dimensiones.


In [2]:
NF=3
Kg = basis(NF,2)
Ka = basis(NF,1)
Ke = basis(NF,0)

# Estado base
psi0 = Kg 

### Para describir los estados de sistemas cuánticos multipartitos (como dos qubits acoplados, un qubit acoplado a un oscilador, etc.) necesitamos expandir el espacio de Hilbert tomando el producto tensorial de los vectores de estado para cada uno de los componentes del sistema. De manera similar, los operadores que actúan sobre los vectores de estado en el espacio de Hilbert combinado (que describe el sistema acoplado) se forman tomando el producto tensorial de los operadores individuales.

In [3]:
Aaa = Ka*Ka.dag()
Aee = Ke*Ke.dag()
Agg = Kg*Kg.dag()
Aeg = Ke*Kg.dag()
Aag = Ka*Kg.dag()
Aea = Ke*Ka.dag()
Aae = Ka*Ke.dag()
Aga = Kg*Ka.dag()
Age = Kg*Ke.dag()

#Valores esperados
ket_a = Aaa #Definiendo los valores esperados con la diagonal principal
ket_e = Aee
ket_g = Agg

expect_values = [Aaa,
Aee,
Agg] #Los valores esperados se ponen en forma de lista, no se si importe su orden


### Parametros del experimento. 

In [39]:
Omega = 3.5 #Frecuencia de rabi
Delta = 0 #Desintonia
ga = 0.015 
gd = 0.05
g = 0.935
ntraj =500 # Numero de trayectorias 
tlist = np.linspace(0,20,10000)

#El numero de trayectorias necesarias para ver periodos brillantes y obscuros tiene que ser 1 
#De lo contrario como ensamble no se ve porque en el mismo coinciden periodos brillantes con los obscuros de otros 
#Esta parpadeo es caracteristico de puntos cuanticos, átomos y algunas moleculas 

### Operadores de colapso

In [40]:
# Operadores de colapso
Ceg = np.sqrt(g)*Age 
Cea = np.sqrt(gd)*Aae
Cag = np.sqrt(ga)*Aga
collapse = [Ceg ,Cea, Cag] #Ceg colapso 0, Cea colapso 1, Cag colapso 2


### Hamiltoniano

In [41]:
#Hamiltoniano del sistema, no se le añaden los terminos de los colapsos porque el programa de la solucion de montecarlo
#lo hace en automatico  con la funcion de liuvillian(H)
H = Delta * Aee + 0.5 * Omega * (Aeg + Age)

### Metodo de montecarlo para resolver las trayectorias cuanticas-

In [ ]:

MC = mcsolve(H, psi0, tlist, collapse, expect_values, ntraj,
             options={'num_cpus': 6})  # Pasar las opciones como diccionario
#La funcion MC para resolver las trayectorias, H es el hamiltoniano, psi0 son las condiciones iniciales.
#tlist es el tiempo con delta t dependiendo de la funcion linspace, collapse los operadores de colapso, expect_values los 
#valores esperados y ntraj el numero de trayectorias.



10.0%. Run time:  40.38s. Est. time left: 00:00:06:03
20.0%. Run time:  81.08s. Est. time left: 00:00:05:24
30.0%. Run time: 122.19s. Est. time left: 00:00:04:45
40.0%. Run time: 163.11s. Est. time left: 00:00:04:04
50.0%. Run time: 204.32s. Est. time left: 00:00:03:24
60.0%. Run time: 245.46s. Est. time left: 00:00:02:43
70.0%. Run time: 286.38s. Est. time left: 00:00:02:02
80.0%. Run time: 327.10s. Est. time left: 00:00:01:21
90.0%. Run time: 368.15s. Est. time left: 00:00:00:40


In [ ]:

#MC = mesolve(H, psi0, tlist, collapse, expect_values)

In [ ]:
collapse = MC.col_which;
times = MC.col_times;
collapse_list = list(zip(collapse[0], times[0]))
lc=collapse_list
data = pd.DataFrame(collapse_list, columns=['colapso','tiempo'])
a=data[data['colapso']==2]
b=data[data['colapso']==1]
c=data[data['colapso']==0]

tamax=len(lc)


## Poblaciones 

In [ ]:

plt.figure(figsize=(8,6))
#plt.bar(tiempos,tdur,width=50,color='orange')

plt.plot(tlist,MC.expect[0], label =r'$\rho_{aa}(t)$', linestyle='-',linewidth = 0.6, color = 'green')
plt.plot(tlist,MC.expect[1], label =r'$\rho_{ee}(t)$', linestyle='-',linewidth = 0.6, color = 'blue')
plt.plot(tlist,MC.expect[2], label =r'$\rho_{gg}(t)$', linestyle='dashdot',linewidth = 0.5, color = 'orange')
plt.xlabel(r'$\gamma t$',fontsize=20)
plt.ylabel(r'$\rho_{ee}(t)$',fontsize=20)
#plt.xlim(0,200)
plt.ylim(0,1)
#plt.legend(loc=1, prop={'size': 25})
#plt.savefig('Fig. 2a.pdf')
#plt.show()

## 